# QSAR modelling in as little lines of code as possible

We often come across ADME or activity datasets and then wish to quickly check how easy or hard it will be for predictive modelling. Recently I came across the package ```MolPipeline``` which integrates ```rdkit``` and ```scikit-learn``` and makes the task of modelling much easier.

I can standardize the dataset, change the features, the ML model to be used, etc by changing a few words.

In [65]:
from molpipeline import Pipeline
from molpipeline.any2mol import AutoToMol
from molpipeline.mol2any import MolToMorganFP, MolToRDKitPhysChem
from molpipeline.mol2mol import (
    ElementFilter,
    SaltRemover,
)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from yellowbrick.regressor import prediction_error, residuals_plot

import pandas as pd

## Import and view dataset:

In [66]:
# import data
ic50 = pd.read_csv("data/pi3k-mtor-pIC50.csv")

In [67]:
#remove molecules with no pIC50:
ic50 = ic50[ic50.pKi_IC50_Pi3K.notnull()]

print(f"There are {ic50.shape[0]} molecules in this dataset")
ic50.head(3)

There are 499 molecules in this dataset


,SMILES,pKi_IC50_Pi3K,pKi_IC50_mTor,Water Solubility,Caco2 permeability,CYP1A2 inhibitor,CYP3A4 inhibitor,Total Clearance
0,Cc1ncnc(-c2ccc(cc2)C(=O)N2CCN3CCCCC3C2)c1C#Cc1...,7.091515,7.769551,-3.540,1.377,0.0,1.0,NaN
1,Cc1ncnc(-c2ccc(cc2)C(C)(C)C#N)c1C#Cc1ccc(N)nc1,7.619789,7.619789,-3.864,1.302,1.0,1.0,0.798
2,Cc1ncnc(-c2ccc(cc2)S(C)(=O)=O)c1C#Cc1ccc(N)nc1,7.187087,8.096910,-3.535,0.274,1.0,1.0,0.899


## Modelling Regression Using Fingerprints

In [68]:
# set up pipeline
pipeline = Pipeline([
      ("auto2mol", AutoToMol()),                                     # reading molecules
      ("element_filter", ElementFilter()),                           # standardization
      ("salt_remover", SaltRemover()),                               # standardization
      ("morgan2_2048", MolToMorganFP(n_bits=2048, radius=2)),        # fingerprints and featurization
      ("RandomForestRegressor", RandomForestRegressor())             # machine learning model
    ],
    n_jobs=4)

#Train-test split
train, test = train_test_split(ic50, test_size=0.1)

# fit the pipeline
pipeline.fit(X=train.SMILES, y=train['pKi_IC50_Pi3K'])

# make predictions on test set:
y_pred = pipeline.predict(test.SMILES)

mse = mean_squared_error(test['pKi_IC50_Pi3K'], y_pred)
r2 = r2_score(test['pKi_IC50_Pi3K'], y_pred)
print(f"MSE: {mse} and r2 score: {r2}")

MSE: 0.107780511001756 and r2 score: 0.45932926665976004


## Modelling Regression Using Phys-Chem Descriptors

In [69]:
# set up pipeline
pipeline_2 = Pipeline([
      ("auto2mol", AutoToMol()),                                     # reading molecules
      ("element_filter", ElementFilter()),                           # standardization
      ("salt_remover", SaltRemover()),                               # standardization
      ("physchem", MolToRDKitPhysChem()),                        # phys-chem descriptors and featurization
      ("RandomForestRegressor", RandomForestRegressor())             # machine learning model
    ],
    n_jobs=4)

#Train-test split
train, test = train_test_split(ic50, test_size=0.1)

# fit the pipeline
pipeline_2.fit(X=train.SMILES, y=train['pKi_IC50_Pi3K'])

# make predictions on test set:
y_pred = pipeline_2.predict(test.SMILES)

mse = mean_squared_error(test['pKi_IC50_Pi3K'], y_pred)
r2 = r2_score(test['pKi_IC50_Pi3K'], y_pred)
print(f"MSE: {mse} and r2 score: {r2}")

MSE: 0.0932056314514693 and r2 score: 0.5610782375000752
